In [ ]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:95% !important; }</style>"))   
%load_ext autoreload
%autoreload 1

Author: Andrew Tarzia

Date Created: 12 Jul 2018

Distributed under the terms of the MIT License.

# Notebook for getting molecule parameters using RDKIT


In [ ]:
import numpy as np
import pandas as pd
import os
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Descriptors3D import NPR1, NPR2, PMI1, PMI2, PMI3
from rdkit.Chem.Draw.MolDrawing import MolDrawing
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Geometry import rdGeometry
from rdkit.Chem import PyMol
from rdkit import Geometry
import tempfile
import matplotlib.pyplot as plt
%aimport rdkit_functions

In [ ]:
# molecule file dir
molecule_file = '/home/atarzia/psp/molecule_param/test_molecules.txt'
# output dir
output_dir = '/home/atarzia/psp/molecule_param/'

In [ ]:
def define_plot_variables(ax, title, ytitle, xtitle, xlim, ylim):
    """
    Series of matplotlib pyplot settings to make all plots unitform.
    """
    # Set number of ticks for x-axis
    ax.tick_params(axis='both', which='major', labelsize=16)

    ax.set_ylabel(ytitle, fontsize=16)
    # ax.legend([y, n], ['aligned', 'not aligned'], loc=4, fancybox=True)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xticklabels(['diffuses', 'does not diffuse'])
    ax.set_xticks([0.25, 0.75])


def define_parity_plot_variables(ax, title, ytitle, xtitle, xlim, ylim):
    """
    Series of matplotlib pyplot settings to make all plots unitform.
    """
    # Set number of ticks for x-axis
    ax.tick_params(axis='both', which='major', labelsize=16)

    ax.set_xlabel(xtitle, fontsize=16)
    ax.set_ylabel(ytitle, fontsize=16)
    # ax.legend([y, n], ['aligned', 'not aligned'], loc=4, fancybox=True)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)


In [ ]:
df, molecules, diameters = rdkit_functions.read_mol_txt_file(molecule_file)
rdkit_functions.produce_quick_fig_mol(molecules, output_dir+'test_molecules.pdf')

## Calculate all MWs

In [ ]:
for m, smile in molecules.items():
    print(m, smile)
    # Read SMILES and add Hs
    mol = Chem.AddHs(Chem.MolFromSmiles(smile))
    MW = Descriptors.MolWt(mol)
    print('MW =', MW)

## Calculate size and shape

In [ ]:
rdkit_functions.calc_molecule_diameters(molecules, 
                                        out_dir=output_dir, vdwScale=0.8, boxMargin=4.0,
                                        spacing=0.6, show_vdw=False, plot_ellip=False,
                                        N_conformers=50, MW_thresh=2000)

## Results

In [ ]:
threshold = 4.2
diffuse = {}
no_diffuse = {}
for name, smile in molecules.items():
    out_file = output_dir+name.replace(' ', '_')+'_diam_result.csv'
    print(out_file)
    if os.path.isfile(out_file) is False:
        continue
    results = pd.read_csv(out_file)
    min_diam = min(results['diam1'])
    mid_diam = min(results['diam2'])
    lit_d = df[df['molecule'] == name]['diffuse'].iloc[0]
    if lit_d == 't':
        if mid_diam <= threshold:
            print(name+':')
            print('can diffuse')
            print('min diameter =', round(min_diam, 3), 'angstrom')
            print('mid diameter =', round(mid_diam, 3), 'angstrom')
            diffuse[name] = smile
        else:
            print(name+':')
            print('cannot diffuse - lit says it can!')
            print('min diameter =', round(min_diam, 3), 'angstrom')
            print('mid diameter =', round(mid_diam, 3), 'angstrom')
            no_diffuse[name] = smile
    else:
        if mid_diam <= threshold:
            print(name+':')
            print('can diffuse - lit says it cannot!')
            print('min diameter =', round(min_diam, 3), 'angstrom')
            print('mid diameter =', round(mid_diam, 3), 'angstrom')
            diffuse[name] = smile
        else:
            print(name+':')
            print('cannot diffuse')
            print('min diameter =', round(min_diam, 3), 'angstrom')
            print('mid diameter =', round(mid_diam, 3), 'angstrom')
            no_diffuse[name] = smile
    print('-')

In [ ]:
if len(diffuse) > 0:
    rdkit_functions.produce_quick_fig_mol(diffuse, output_dir+'diffuse_mol.pdf', 
                                          labels=False, mpr=5, ims=200)
if len(no_diffuse) > 0:
    rdkit_functions.produce_quick_fig_mol(no_diffuse, output_dir+'no_diffuse_mol.pdf', 
                                          labels=False, mpr=5, ims=200)

## Plot parity of kinetic diameters

In [ ]:
threshold = 4.2
fig, ax = plt.subplots(figsize=(5,5))
for name, smile in molecules.items():
    try:
        kin_diam = float(diameters[name])
    except ValueError:
        print('no radius given for this molecule - skipped')
        continue
    out_file = output_dir+name.replace(' ', '_')+'_diam_result.csv'
    if os.path.isfile(out_file) is False:
        continue
    results = pd.read_csv(out_file)
    mid_diam = min(results['diam2'])
    lit_d = df[df['molecule'] == name]['diffuse'].iloc[0]
    if lit_d == 't':
        if mid_diam <= threshold:
            C = 'b'
            M = 'o'
            E = 'k'
            D = 0.25
        else:
            C = 'b'
            M = 'X'
            E = 'k'
            D = 0.25
    else:
        if mid_diam <= threshold:
            C = 'r'
            M = 'X'
            E = 'k'
            D = 0.75
        else:
            C = 'r'
            M = 'o'
            E = 'k'
            D = 0.75
    ax.scatter(kin_diam, mid_diam, c=C, 
               edgecolors=E, marker=M, alpha=1.0,
               s=80)
    
ax.axhline(y=threshold, c='k')
ax.axvline(x=threshold, c='k')
ax.plot(np.linspace(-1, 12, 2), np.linspace(-1, 12, 2), c='k', alpha=0.4)
# plot the limit from the two Sholl papers on diffusion
# ax.axvspan(4.0, 4.2, facecolor='r', alpha=0.5)


define_parity_plot_variables(ax,
                             title='',
                             xtitle='kinetic diameter [$\mathrm{\AA}$]',
                             ytitle='intermediate diameter [$\mathrm{\AA}$]',
                             xlim=(0, 10),
                             ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"parity.pdf", dpi=720,
            bbox_inches='tight')

## Plot categorical scatter

In [ ]:
threshold = 4.2
dx = 0.15
fig, ax = plt.subplots(figsize=(5,5))
for name, smile in molecules.items():
    out_file = output_dir+name.replace(' ', '_')+'_diam_result.csv'
    if os.path.isfile(out_file) is False:
        continue
    results = pd.read_csv(out_file)
    mid_diam = min(results['diam2'])
    lit_d = df[df['molecule'] == name]['diffuse'].iloc[0]
    if lit_d == 't':
        if mid_diam <= threshold:
            C = 'b'
            M = 'o'
            E = 'k'
            D = 0.25
        else:
            C = 'b'
            M = 'X'
            E = 'k'
            D = 0.25
    else:
        if mid_diam <= threshold:
            C = 'r'
            M = 'X'
            E = 'k'
            D = 0.75
        else:
            C = 'r'
            M = 'o'
            E = 'k'
            D = 0.75
    ax.scatter(D+(dx*(np.random.random() - 0.5) * 2),
               mid_diam, c=C, 
               edgecolors=E, marker=M, alpha=1.0,
               s=80)
    
ax.axhline(y=threshold, c='k')
define_plot_variables(ax,
                      title='',
                      xtitle='',
                      ytitle='intermediate diameter [$\mathrm{\AA}$]',
                      xlim=(0, 1),
                      ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"categorical.pdf", dpi=720,
            bbox_inches='tight')

## Plot Shapes

In [ ]:
threshold = 4.2
fig, ax = plt.subplots(figsize=(5,5))
for name, smile in molecules.items():
    out_file = output_dir+name.replace(' ', '_')+'_diam_result.csv'
    if os.path.isfile(out_file) is False:
        continue
    results = pd.read_csv(out_file)
    mid_diam = min(results['diam2'])
    lit_d = df[df['molecule'] == name]['diffuse'].iloc[0]
    if lit_d == 't':
        if mid_diam <= threshold:
            C = 'b'
            M = 'o'
            E = 'k'
            D = 0.25
        else:
            C = 'b'
            M = 'X'
            E = 'k'
            D = 0.25
    else:
        if mid_diam <= threshold:
            C = 'r'
            M = 'X'
            E = 'k'
            D = 0.75
        else:
            C = 'r'
            M = 'o'
            E = 'k'
            D = 0.75
    ax.scatter(np.average(results['ratio_1']),
               np.average(results['ratio_2']),
               c=C, 
               edgecolors=E, marker=M, alpha=1.0,
               s=80)
    
ax.plot([0, 0.5, 1, 0], [1, 0.5, 1, 1], c='k', lw=2)
ax.text(0.75, 1.03, 'sphere', fontsize=20)
ax.text(0.4, 0.45, 'oblate', fontsize=20)
ax.text(-0.05, 1.03, 'prolate', fontsize=20)
define_parity_plot_variables(ax,
                             title='',
                             xtitle='$I_1$ / $I_3$',
                             ytitle='$I_2$ / $I_3$',
                             xlim=(-0.1, 1.1),
                             ylim=(0.4, 1.1))
fig.tight_layout()
fig.savefig(output_dir+"shape.pdf", dpi=720,
            bbox_inches='tight')

## Spacing tests:  -- done 18/7/18
- run ellipsoid tests 25 times with 10 conformers each time and save the average and std deviation of the min and mid diameter

In [ ]:
# molecule: {spacing: (avg, std)}
spacing_tests_min = {'benzene': {'0.1': (0, 0),
                                 '0.2': (0, 0),
                                 '0.4': (3.3794289152145467 , 0.019673085424443428),
                                 '0.5': (3.3333897302564544 , 0.028678701518417928),
                                 '0.6': (3.2315745363725337 , 0.017849243469717994),
                                 '1.0': (2.849180995220388 , 0.04838637474340038)},
                     'n-hexane': {'0.1': (0, 0),
                                  '0.2': (0, 0),
                                  '0.4': (5.0835923862674495 , 0.06895603460636741),
                                  '0.5': (4.978275495360503 , 0.05683250141249099),
                                  '0.6': (4.873315126271812 , 0.06108305615909574),
                                  '1.0': (4.497290421763366 , 0.08750991331271302)},
                     '1,2,4-trimethylbenzene': {'0.1': (0, 0),
                                                '0.3': (4.840076609847893 , 0.045205419877336145),
                                                '0.4': (4.74736322402256 , 0.03994801793792416),
                                                '0.5': (4.641228188625925 , 0.06616635800790348),
                                                '0.6': (4.488226784532858 , 0.04847658400864577),
                                                '1.0': (3.9248371015184085 , 0.12679311000462484)}}

spacing_tests_mid = {'benzene': {'0.1': (0, 0),
                                 '0.2': (0, 0),
                                 '0.4': (6.649454451273829 , 0.013495748028405818),
                                 '0.5': (6.546971715975721 , 0.037964346782573745),
                                 '0.6': (6.525331194565837 , 0.029609796662037183),
                                 '1.0': (6.068867263161149 , 0.07011555279897945)},
                     'n-hexane': {'0.1': (0, 0),
                                  '0.2': (0, 0),
                                  '0.4': (5.727257044536465 , 0.1284078518182792),
                                  '0.5': (5.670823148829275 , 0.11718708499887756),
                                  '0.6': (5.5647520341975625 , 0.10709935627025727),
                                  '1.0': (5.32170216216025 , 0.13483968819479367)},
                     '1,2,4-trimethylbenzene': {'0.1': (0, 0),
                                                '0.3': (7.426353842175533 , 0.03786939839775834),
                                                '0.4': (7.333573135450377 , 0.04630508640316625),
                                                '0.5': (7.314738056193096 , 0.048482526890533326),
                                                '0.6': (7.226540741526308 , 0.07419202615800291),
                                                '1.0': (7.013051712007243 , 0.07598138464225354)}}
                     
markers = {'benzene': 'o', 'n-hexane': 'X', '1,2,4-trimethylbenzene': 'D'}
colours = {'benzene': 'k', 'n-hexane': 'r', '1,2,4-trimethylbenzene': 'b'}

In [ ]:
fig, ax = plt.subplots()
for mol in spacing_tests_min.keys():
    for key, val in spacing_tests_min[mol].items():
        sp = float(key)
        avg = val[0]
        std = val[1]
        ax.errorbar(sp, avg, c=colours[mol], yerr=std, fmt=markers[mol])

define_parity_plot_variables(ax,
                             title='',
                             xtitle='spacing [$\AA$]',
                             ytitle='min diameter [$\AA$]',
                             xlim=(0, 1.1),
                             ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"min_spacing.pdf", dpi=720,
            bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
for mol in spacing_tests_mid.keys():
    for key, val in spacing_tests_mid[mol].items():
        sp = float(key)
        avg = val[0]
        std = val[1]
        ax.errorbar(sp, avg, c=colours[mol], yerr=std, fmt=markers[mol])

define_parity_plot_variables(ax,
                             title='',
                             xtitle='spacing [$\AA$]',
                             ytitle='mid diameter [$\AA$]',
                             xlim=(0, 1.1),
                             ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"mid_spacing.pdf", dpi=720,
            bbox_inches='tight')

## Conformer No. Tests - done 18/7/18
- 2 trials
- differ number of conformers
- spacing = 0.6

In [ ]:
# molecule: {spacing: (avg, std)}
conformer_tests_min = {'benzene': {'1': (3.221373248755991 , 0.004852646654566861),
                                   '10': (3.2588304836940942 , 0.006956539037165754),
                                   '50': (3.2436940191596317 , 0.016198271266676167),
                                   '100': (3.2326218504059288 , 0.003853874440666649),
                                   '500': (3.2330473832007325 , 0.0018236559425053311)},
                     'n-hexane': {'1': (5.141141045458199 , 0.13304093800965688),
                                   '10': (4.957373679413936 , 0.023610168570660228),
                                   '50': (4.87212020107353 , 0.017040469280273474),
                                   '100': (4.911133486971741 , 0.01723649729619181),
                                   '500': (4.891902480769925 , 0.008404023636215285)},
                     '1,2,4-trimethylbenzene': {'1': (4.498871204493703 , 0.1362145494522462),
                                   '10': (4.478884245679016 , 0.015539006468946681),
                                   '50': (4.507169342491871 , 0.0035833227709809456),
                                   '100': (4.487331546112546 , 0.006343296733285264),
                                   '500': (4.493251428144968 , 0.0030979508775335063)}}

conformer_tests_mid = {'benzene': {'1': (6.473960279275529 , 0.024503187415663596),
                                   '10': (6.501792400073141 , 0.01924244928963903),
                                   '50': (6.528516395372385 , 0.008131803260291726),
                                   '100': (6.5407585651138795 , 0.002276161285552636),
                                   '500': (6.53315674537609 , 0.0012551802254567868)},
                     'n-hexane': {'1': (5.248219801680463 , 0.1596778847869671),
                                   '10': (5.512843592215781 , 0.00251194113303832),
                                   '50': (5.595576007199124 , 0.09552089437927647),
                                   '100': (5.576820296115795 , 0.04429024204647147),
                                   '500': (5.581235665531599 , 0.007401442176630635)},
                     '1,2,4-trimethylbenzene': {'1': (7.36604554182822 , 0.04078536880172745),
                                   '10': (7.276754072294647 , 0.052813283959488366),
                                   '50': (7.214263004894109 , 0.017593374853946298),
                                   '100': (7.229715994297686 , 0.003505550538929647),
                                   '500': (7.239862461826757 , 0.001030076037960903)}}
                     
markers = {'benzene': 'o', 'n-hexane': 'X', '1,2,4-trimethylbenzene': 'D'}
colours = {'benzene': 'k', 'n-hexane': 'r', '1,2,4-trimethylbenzene': 'b'}

In [ ]:
fig, ax = plt.subplots()
for mol in conformer_tests_min.keys():
    for key, val in conformer_tests_min[mol].items():
        sp = float(key)
        avg = val[0]
        std = val[1]
        ax.errorbar(sp, avg, c=colours[mol], yerr=std, fmt=markers[mol])

define_parity_plot_variables(ax,
                             title='',
                             xtitle='$N$',
                             ytitle='min diameter [$\AA$]',
                             xlim=(0, 550),
                             ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"min_conf.pdf", dpi=720,
            bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots()
for mol in conformer_tests_mid.keys():
    for key, val in conformer_tests_mid[mol].items():
        sp = float(key)
        avg = val[0]
        std = val[1]
        ax.errorbar(sp, avg, c=colours[mol], yerr=std, fmt=markers[mol])

define_parity_plot_variables(ax,
                             title='',
                             xtitle='$N$',
                             ytitle='mid diameter [$\AA$]',
                             xlim=(0, 550),
                             ylim=(0, 10))
fig.tight_layout()
fig.savefig(output_dir+"mid_conf.pdf", dpi=720,
            bbox_inches='tight')